In [1]:
import numpy as np
import pandas as pd
import itertools
from pydantic import BaseModel
from enum import Enum, auto
import random
from typing import Optional

In [2]:
from maverick import Deck, Holding, Hand

deck = Deck.build().shuffle()
holding = Holding(cards=deck.deal(2))
hand = Hand(private_cards=holding.cards, community_cards=deck.deal(3))

In [3]:
hand, hand.score()

(Hand(private_cards=[Card(suit=<Suit.SPADES: 'S'>, rank=<Rank.SEVEN: 7>), Card(suit=<Suit.SPADES: 'S'>, rank=<Rank.JACK: 11>)], community_cards=[Card(suit=<Suit.CLUBS: 'C'>, rank=<Rank.ACE: 14>), Card(suit=<Suit.HEARTS: 'H'>, rank=<Rank.NINE: 9>), Card(suit=<Suit.DIAMONDS: 'D'>, rank=<Rank.JACK: 11>)]),
 (<HandType.PAIR: 'pair'>, 26.1497))

In [4]:
deck = Deck.standard_deck(shuffle=True)
all_possible_holdings = list(Holding.all_possible_holdings(deck.cards, n=2))
all_possible_holdings[0]

Holding(cards=[Card(suit=<Suit.CLUBS: 'C'>, rank=<Rank.KING: 13>), Card(suit=<Suit.HEARTS: 'H'>, rank=<Rank.KING: 13>)])

In [6]:
deck = Deck.standard_deck(shuffle=True)
all_possible_hands = list(Hand.all_possible_hands(deck.cards))
all_possible_hands[0]

Hand(private_cards=[Card(suit=<Suit.SPADES: 'S'>, rank=<Rank.ACE: 14>), Card(suit=<Suit.CLUBS: 'C'>, rank=<Rank.JACK: 11>)], community_cards=[Card(suit=<Suit.SPADES: 'S'>, rank=<Rank.QUEEN: 12>), Card(suit=<Suit.CLUBS: 'C'>, rank=<Rank.QUEEN: 12>), Card(suit=<Suit.CLUBS: 'C'>, rank=<Rank.ACE: 14>)])

In [7]:
[hand.score() for hand in all_possible_hands[:5]]

[(<HandType.TWO_PAIR: 'two_pair'>, 44.131),
 (<HandType.PAIR: 'pair'>, 27.152),
 (<HandType.PAIR: 'pair'>, 27.1518),
 (<HandType.PAIR: 'pair'>, 27.1519),
 (<HandType.TWO_PAIR: 'two_pair'>, 44.131)]

In [11]:
class Suit(Enum):
    HEARTS = 'H'
    SPADES = 'S'
    CLUBS = 'C'
    DIAMONDS = 'D'
    

class Rank(Enum):
    TWO = 2
    THREE = 3
    FOUR = 4
    FIVE = 5
    SIX = 6
    SEVEN = 7
    EIGHT = 8
    NINE = 9
    TEN = 10
    JACK = 11
    QUEEN = 12
    KING = 13
    ACE = 14


class Street(Enum):
    PRE_FLOP = 0
    FLOP = 1
    TURN = 2
    RIVER = 3
    SHOWDOWN = 4
    

class Card(BaseModel):
    suit: Suit
    rank: Rank
    
    @classmethod
    def random(cls, n: int = 1) -> list["Card"]:
        suits = list(Suit)
        ranks = list(Rank)
        all_cards = [cls(suit=s, rank=r) for s in suits for r in ranks]
        selected = random.sample(all_cards, n)
        return selected if n > 1 else selected[0]


class Deck(BaseModel):
    cards: list[Card]

    @classmethod
    def build(cls):
        ranks = list(Rank)
        suits = list(Suit)
        cards = []
        for rank in ranks:
            for suit in suits:
                card = Card(suit=suit, rank=rank)
                cards.append(card)
        return cls(cards=cards)
    
    def deal(self, n: int) -> list[Card]:
        """Deal n random cards from the deck."""
        dealt_cards = random.sample(self.cards, n)
        for card in dealt_cards:
            self.cards.remove(card)
        return dealt_cards


class Holding(BaseModel):
    """Two cards held by a player."""
    cards: list[Card]

    @classmethod
    def random(cls, deck: Optional[Deck]=None):
        if deck:
            cards = random.sample(deck.cards, 2)
        else:
            cards = Card.random(2)
        return cls(cards=cards)


class Hand(BaseModel):
    """Private cards plus as many community cards as needed to complete the hand."""
    private_cards: list[Card]
    community_cards: list[Card]
    
    def score(self) -> int:
        """Score the hand using some poker hand evaluation logic."""
        all_cards = self.private_cards + self.community_cards
        assert len(all_cards) == 5, "Exactly 5 cards are required to score a hand."
        # Placeholder for actual scoring logic
        pass
    

class Player(BaseModel):
    holding: Holding
    is_dealer: bool = False
    is_small_blind: bool = False
    is_big_blind: bool = False
    stack: int = 0
    

class Game(BaseModel):
    players: list[Player]
    community_cards: list[Card]
    deck: Deck
    small_blind_amount: int = 0
    big_blind_amount: int = 0
    pot: int = 0

In [12]:
def build_deck():
    numbers = list(range(2,15))
    suits = ['H','S','C','D']
    deck = []
    for i in numbers:
        for s in suits:
            card = s+str(i)
            deck.append(card)
    return deck


def combinations(arr, n):
    arr = np.asarray(arr)
    t = np.dtype([('', arr.dtype)]*n)
    result = np.fromiter(itertools.combinations(arr, n), t)
    return result.view(arr.dtype).reshape(-1, n)


def check_four_of_a_kind(hand,letters,numbers,rnum,rlet):
    for i in numbers:
        if numbers.count(i) == 4:
            four = i
        elif numbers.count(i) == 1:
            card = i
    score = 105 + four + card/100
    return score


def check_full_house(hand,letters,numbers,rnum,rlet):
    for i in numbers:
        if numbers.count(i) == 3:
            full = i
        elif numbers.count(i) == 2:
            p = i
    score = 90 + full + p/100
    return score


def check_three_of_a_kind(hand,letters,numbers,rnum,rlet):
    cards = []
    for i in numbers:
        if numbers.count(i) == 3:
            three = i
        else:
            cards.append(i)
    score = 45 + three + max(cards) + min(cards)/1000
    return score


def check_two_pair(hand,letters,numbers,rnum,rlet):
    pairs = []
    cards = []
    for i in numbers:
        if numbers.count(i) == 2:
            pairs.append(i)
        elif numbers.count(i) == 1:
            cards.append(i)
            cards = sorted(cards,reverse=True)
    score = 30 + max(pairs) + min(pairs)/100 + cards[0]/1000
    return score


def check_pair(hand,letters,numbers,rnum,rlet):
    pair = []
    cards = []
    for i in numbers:
        if numbers.count(i) == 2:
            pair.append(i)
        elif numbers.count(i) == 1:
            cards.append(i)
            cards = sorted(cards,reverse=True)
    score = 15 + pair[0] + cards[0]/100 + cards[1]/1000 + cards[2]/10000
    return score


def score_hand(hand):
    # We get the suit for each card in the hand
    letters = [hand[i][:1] for i in range(5)]
    
    # We get the number for each card in the hand
    numbers = [int(hand[i][1:]) for i in range(5)]
    
    # We count repetitions for each number
    rnum = [numbers.count(i) for i in numbers]
    
    # We count repetitions for each letter
    rlet = [letters.count(i) for i in letters]
    
    # The difference between the greater and smaller number in the hand
    dif = max(numbers) - min(numbers)
    
    handtype = ''
    score = 0
    if 5 in rlet:
        if numbers == [14,13,12,11,10]:
            handtype = 'royal_flush'
            score = 135
            # print('this hand is a %s:, with score: %s' % (handtype,score))
            # I comment the prints so the script runs faster
        elif dif == 4 and max(rnum) == 1:
            handtype = 'straight_flush'
            score = 120 + max(numbers)
            hand2 = ['H7', 'H7', 'H7', 'H7', 'H7']
            # print('this hand is a %s:, with score: %s' % (handtype,score))
        elif 4 in rnum:
            handtype == 'four of a kind'
            score = check_four_of_a_kind(hand,letters,numbers,rnum,rlet)
            # print('this hand is a %s:, with score: %s' % (handtype,score))
        elif sorted(rnum) == [2,2,3,3,3]:
            handtype == 'full house'
            score = check_full_house(hand,letters,numbers,rnum,rlet)
            # print('this hand is a %s:, with score: %s' % (handtype,score))
        elif 3 in rnum:
            handtype = 'three of a kind'
            score = check_three_of_a_kind(hand,letters,numbers,rnum,rlet)
            # print('this hand is a %s:, with score: %s' % (handtype,score))
        elif rnum.count(2) == 4:
            handtype = 'two pair'
            score = check_two_pair(hand,letters,numbers,rnum,rlet)
            # print('this hand is a %s:, with score: %s' % (handtype,score))
        elif rnum.count(2) == 2:
            handtype = 'pair'
            score = check_pair(hand,letters,numbers,rnum,rlet)
            # print('this hand is a %s:, with score: %s' % (handtype,score))
        else:
            handtype = 'flush'
            score = 75 + max(numbers)/100
            # print('this hand is a %s:, with score: %s' % (handtype,score))
    elif 4 in rnum:
        handtype = 'four of a kind'
        score = check_four_of_a_kind(hand,letters,numbers,rnum,rlet)
        # print('this hand is a %s:, with score: %s' % (handtype,score))
    elif sorted(rnum) == [2,2,3,3,3]:
        handtype = 'full house'
        score = check_full_house(hand,letters,numbers,rnum,rlet)
        # print('this hand is a %s:, with score: %s' % (handtype,score))
    elif 3 in rnum:
        handtype = 'three of a kind'
        score = check_three_of_a_kind(hand,letters,numbers,rnum,rlet)
        # print('this hand is a %s:, with score: %s' % (handtype,score))
    elif rnum.count(2) == 4:
        handtype = 'two pair'
        score = check_two_pair(hand,letters,numbers,rnum,rlet)
        # print('this hand is a %s:, with score: %s' % (handtype,score))
    elif rnum.count(2) == 2:
        handtype = 'pair'
        score = check_pair(hand,letters,numbers,rnum,rlet)
        # print('this hand is a %s:, with score: %s' % (handtype,score))
    elif dif == 4:
        handtype = 'straight'
        score = 65 + max(numbers)
        # print('this hand is a %s:, with score: %s' % (handtype,score))

    else:
        handtype = 'high card'
        n = sorted(numbers,reverse=True)
        score = n[0] + n[1]/100 + n[2]/1000 + n[3]/10000 + n[4]/100000
        # print('this hand is a %s:, with score: %s' % (handtype,score))

    return score


def handvalues(combinations):
    # We iterate over all combinations scoring them
    scores = [{"hand": i, "value": score_hand(i)} for i in combinations]
    # We sort hands by score
    scores = sorted(scores, key = lambda k: k['value'])
    return scores


In [13]:
# We create our deck
deck = build_deck()

# We create an array containing all possible 5 cards combinations
combs = combinations(deck, 5)
hand_values = handvalues(combs)

In [17]:
combs

array([['H2', 'S2', 'C2', 'D2', 'H3'],
       ['H2', 'S2', 'C2', 'D2', 'S3'],
       ['H2', 'S2', 'C2', 'D2', 'C3'],
       ...,
       ['C13', 'D13', 'S14', 'C14', 'D14'],
       ['C13', 'H14', 'S14', 'C14', 'D14'],
       ['D13', 'H14', 'S14', 'C14', 'D14']],
      shape=(2598960, 5), dtype='<U3')

In [15]:
score_hand(['H14', 'H13', 'H12', 'H11', 'H10'])

135

In [16]:
# making a list of hands
x = [i.get("hand","") for i in hand_values]

# making a list of values
y = [i.get("value","") for i in hand_values]

data = {'hands':x, 'value':y}  # making a dictionary of hands and values

df = pd.DataFrame(data)  # making a pandas dataframe with hands and values
df

,hands,value
0,"[H2, H3, H4, H5, S7]",7.05432
1,"[H2, H3, H4, H5, C7]",7.05432
2,"[H2, H3, H4, H5, D7]",7.05432
3,"[H2, H3, H4, S5, H7]",7.05432
4,"[H2, H3, H4, S5, S7]",7.05432
...,...,...
2598955,"[D9, D10, D11, D12, D13]",133.00000
2598956,"[H10, H11, H12, H13, H14]",134.00000
2598957,"[S10, S11, S12, S13, S14]",134.00000
2598958,"[C10, C11, C12, C13, C14]",134.00000
